You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [9]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [10]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [11]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [12]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [13]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
        "3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [14]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [15]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit], # the order matters
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan Document

Title: Unveiling the Latest Trends and Key Players in Artificial Intelligence

Outline:

I. Introduction
- Definition of Artificial Intelligence
- Importance of AI in today's digital world
- Brief overview of the latest trends in AI

II. Latest Trends in Artificial Intelligence
A. Machine Learning
- Advancements in deep learning algorithms
- Applications in various industries such as healthcare and finance
- Impact on automation and decision-making processes

B. Natural Language Proces

I now can give a great answer

Final Answer: 

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become an integral part of today's digital world, revolutionizing industries and shaping the future of technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in driving innovation and automation. The latest trends in AI encompass advancements in Machine Learning, Natural Language Processing, and Ethical AI, each contributing to the growth and development of intelligent systems.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine Learning, a subset of AI, has seen significant progress with advancements in deep learning algorithms. These algorithms have found applications across various industries such as healthcare and finance, enabling predictive analytics and personalized services. The impact of Machine Learning extends to automation and decisi

- Display the results of your execution as markdown in the notebook.

In [17]:
from IPython.display import Markdown
Markdown(result)

# Unveiling the Latest Trends and Key Players in Artificial Intelligence

## Introduction
Artificial Intelligence (AI) has become an integral part of today's digital world, revolutionizing industries and shaping the future of technology. Defined as the simulation of human intelligence processes by machines, AI plays a crucial role in driving innovation and automation. The latest trends in AI encompass advancements in Machine Learning, Natural Language Processing, and Ethical AI, each contributing to the growth and development of intelligent systems.

## Latest Trends in Artificial Intelligence

### Machine Learning
Machine Learning, a subset of AI, has seen significant progress with advancements in deep learning algorithms. These algorithms have found applications across various industries such as healthcare and finance, enabling predictive analytics and personalized services. The impact of Machine Learning extends to automation and decision-making processes, transforming how businesses operate and make strategic decisions.

### Natural Language Processing
Natural Language Processing (NLP) focuses on the interaction between computers and human language, leading to the development of conversational AI. This technology is utilized in customer service and virtual assistants, enhancing user experience and communication. The future implications of NLP in human-computer interaction are vast, with potential advancements in voice recognition and language translation.

### Ethical AI
Ethical considerations in AI have gained prominence, with discussions on bias and fairness in AI algorithms. Regulatory measures are being implemented to ensure ethical AI implementation, emphasizing transparency and accountability in AI systems. As AI continues to evolve, maintaining ethical standards is crucial to building trust and reliability in intelligent technologies.

## Key Players in Artificial Intelligence

### Tech Giants
Tech giants like Google, Microsoft, and IBM are leading the way in AI research and development. Google's AI research division, Google Brain, focuses on advancing machine learning capabilities, while Microsoft's Azure Cognitive Services provide AI solutions for businesses. IBM Watson is known for its cognitive computing technology, transforming industries with AI-powered insights.

### AI Startups
AI startups such as DeepMind, OpenAI, and UiPath are disrupting the industry with innovative solutions. DeepMind specializes in deep learning and reinforcement learning, pushing the boundaries of AI research. OpenAI focuses on developing AI for the betterment of humanity, while UiPath offers robotic process automation for businesses seeking efficiency and productivity.

### Academic Institutions
Leading academic institutions like Stanford University, MIT, and the University of Toronto have established AI labs and research centers. Stanford University's AI Lab conducts cutting-edge research in machine learning and computer vision, while MIT's CSAIL pioneers breakthroughs in AI and robotics. The University of Toronto's Vector Institute is dedicated to advancing AI research and education, shaping the future of intelligent systems.

## Noteworthy News in Artificial Intelligence
Stay informed about recent breakthroughs in AI research, industry partnerships, and collaborations driving innovation in AI. Explore the impact of AI on the job market and future workforce, as industries adapt to the changing landscape of intelligent technologies. Follow our blog for regular updates and insights on the latest news in Artificial Intelligence.

## Conclusion
Artificial Intelligence continues to evolve and shape the digital landscape, with advancements in Machine Learning, Natural Language Processing, and Ethical AI driving innovation. Key players in AI, including tech giants, startups, and academic institutions, are leading the way in research and development. Stay informed about the latest trends and developments in AI by following our blog for valuable insights and updates on the future of Artificial Intelligence.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [18]:
topic = "Pliego (Murcia)"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Pliego (Murcia).
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer:

Content Plan: 
Topic: Pliego (Murcia) Blog Article

I. Introduction
- Brief overview of Pliego (Murcia) and its significance
- Mention of latest trends and key players in the region

II. Key Points
A. Historical Background of Pliego
- Insight into the history and cultural heritage of the town
- Notable landmarks and attractions

B. Economy and Industry in Pliego
- Overview of the main economic activities in the region
- Key players and industries driving the local economy

C. Tourism and Leisure in Pliego
- Highlight

I now can give a great answer

Final Answer:
# Discovering the Hidden Gem of Pliego (Murcia)

## Introduction
Nestled in the heart of Murcia, Pliego stands as a hidden gem waiting to be explored. With a rich historical background and a vibrant community, this town offers a unique blend of tradition and modernity. In recent years, Pliego has been gaining attention for its growing tourism industry and the development of its economy, making it a promising destination for travelers and investors alike.

## Historical Background of Pliego
Pliego boasts a fascinating history that dates back centuries, reflected in its well-preserved landmarks and cultural heritage. From the iconic Castle of Pliego to the charming streets lined with historic buildings, every corner of the town tells a story of its past. Visitors can immerse themselves in the rich tapestry of Pliego's history, exploring ancient ruins and monuments that speak volumes about its heritage.

## Economy and Industry in Pliego
While 

In [19]:
Markdown(result)

# Discovering the Hidden Gem of Pliego (Murcia)

## Introduction
Nestled in the heart of Murcia, Pliego stands as a hidden gem waiting to be explored. With a rich historical background and a vibrant community, this town offers a unique blend of tradition and modernity. In recent years, Pliego has been gaining attention for its growing tourism industry and the development of its economy, making it a promising destination for travelers and investors alike.

## Historical Background of Pliego
Pliego boasts a fascinating history that dates back centuries, reflected in its well-preserved landmarks and cultural heritage. From the iconic Castle of Pliego to the charming streets lined with historic buildings, every corner of the town tells a story of its past. Visitors can immerse themselves in the rich tapestry of Pliego's history, exploring ancient ruins and monuments that speak volumes about its heritage.

## Economy and Industry in Pliego
While deeply rooted in tradition, Pliego is also a town on the move, with a dynamic economy driven by various industries. Agriculture plays a significant role in the region, with olive groves and vineyards dotting the landscape. Additionally, the town is home to a thriving artisanal sector, producing traditional crafts that are highly sought after. With a focus on sustainable development, Pliego is poised for growth and innovation in the years to come.

## Tourism and Leisure in Pliego
For travelers seeking an authentic Spanish experience, Pliego offers a plethora of opportunities for exploration and relaxation. From hiking trails in the Sierra Espuña Natural Park to cultural events celebrating local traditions, there is something for everyone in this charming town. Visitors can also indulge in the region's culinary delights, savoring the flavors of traditional dishes made with fresh, local ingredients.

## Community and Lifestyle in Pliego
At the heart of Pliego lies its warm and welcoming community, known for its hospitality and festive spirit. Throughout the year, the town comes alive with cultural events, festivals, and gastronomic fairs that showcase the best of local traditions. Residents and visitors alike can partake in these celebrations, experiencing firsthand the vibrant and lively atmosphere that defines life in Pliego.

## Call to Action
As you plan your next travel adventure or business venture, consider adding Pliego to your list of destinations. Whether you're a history buff, a nature enthusiast, or a foodie in search of new flavors, this town has something special to offer. Come and discover the magic of Pliego for yourself, and immerse yourself in an experience that will stay with you long after you leave.

This blog post aims to shed light on the hidden gem of Pliego (Murcia) and showcase the town's unique blend of history, culture, and community. With a focus on providing valuable insights and information to a diverse audience, from travel enthusiasts to potential investors, this article invites readers to explore the wonders of Pliego and experience all that this charming town has to offer.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).